In [ ]:
from sklearn.model_selection import train_test_split



# gets the training and test file names from the command line
parser = argparse.ArgumentParser()
parser.add_argument("train", nargs='?', default="graduate-project-dalcantara7/2018-E-c-En-train.txt")
parser.add_argument("test", nargs='?', default="graduate-project-dalcantara7/2018-E-c-En-test.txt")
args = parser.parse_args()

# reads train and dev data into Pandas data frames
read_csv_kwargs = dict(sep="\t",
                        converters={e: emotion_to_int.get for e in emotions})
train_data = pd.read_csv(args.train, **read_csv_kwargs)
test_data = pd.read_csv(args.test, **read_csv_kwargs)

clean_tweets(train_data['Tweet'], test_data['Tweet'])

train_word_indices, test_word_indices, embedding_layer = preprocess_data(train_data, test_data)

train_labels = np.asarray(train_data.iloc[:,2:13]) # get labels
test_labels = np.asarray(test_data.iloc[:,2:13]) # get labels

# makes predictions on the dev set
test_predictions = train_and_predict(train_word_indices, train_labels, test_word_indices, test_labels, NUM_WORDS, test_data, embedding_layer)

# prints out multi-label accuracy
print("accuracy: {:.3f}".format(sklearn.metrics.jaccard_similarity_score(
    test_data[emotions], test_predictions[emotions])))

# saves predictions and creates submission zip file
test_predictions.to_csv("graduate-project-dalcantara7/E-C_en_pred.txt", sep="\t", index=False)
with zipfile.ZipFile('graduate-project-dalcantara7/submission.zip', mode='w') as submission_zip:
    submission_zip.write("graduate-project-dalcantara7/E-C_en_pred.txt")
